# Advanced Pandas Assignment 3

In this assignment, you will practice cleaning "dirty" data.

### Note about assignments
You can add lines of code according to your preferences. As long as the code required by the assignment is found in this notebook under the corresponding question header (ie. the answer to question 1 is underneath the title "Question 1"), you will receive credit for it.

## About the data
The data used in this assignment is a table built from the Human Resources schema of the Adventure Works 2019 database. This data contains information about each time that Employee Pay History was changed (each line is a pay rate change). It also contains information about the employee and the department they were working in when they received the pay rate listed.

This data is similar to data used in previous assignments but has been "dirtied" in order to make this assignment possible. 

The actual data is stored in a CSV file located inside the `data` folder. The file is called `pay_history.csv`.

## Instructions
### Set up
##### Import Pandas
Import the Pandas library into Jupyter Lab.

In [ ]:
import pandas as pd

##### Disable column display limit
Use the following code to disable the default limit for displaying columns. If you don't use this code, a data set with more than 20 columns will be truncated when displayed to take up less space.

```
pd.options.display.max_columns = None
```

In [ ]:
pd.options.display.max_columns = None

##### Create the dataframe
Use the `read_csv()` function from Pandas to read the data from the `pay_history.csv` file into a dataframe called `df`.

In [ ]:
df = pd.read_csv("./data/pay_history.csv")

##### Preview dataframe
Use the `.head()` method to print out the first 5 rows of the dataframe.

In [ ]:
df.head()

### Questions
The data you were given to analyze looks perfectly normal at first glance. However, knowing that humans are imperfect at inputting data, you decide to do a quick glance over the data to make sure that nothing strange is happening. If there are mistakes in the data, you are determined to fix them in such a way that they don't affect the analysis.


#### Find Null Values
##### Question 1: Describe the data 
Before anything else, use the `.info()` method to check the count of null values in the data set.

In [ ]:
df.info()

##### Question 2: Count null values
Use the `.isna()` method in conjunction with the `.sum()` method to get a count of null values in each column.

In [ ]:
df.isna().sum()

##### Question 3: Which columns have null values?
Which columns have null values? What do you think should be done to these columns to prepare the data set for analysis (imputation or drop the column)?

1. Rate - Imputation
2. OrganizationLevel - Imputation
3. SalariedFlag - Drop Column
4. EndDate - Drop Column
5. Sub-Department - Imputation

#### Rate - Imputation (part 1)
Because there are only two missing values in the `Rate` column, let's try to impute them. We can start to do this by discovering the average rate and then assigning it to both rows where `Rate` is null.

##### Question 4: Select the `Rate` column
Get the `Rate` column out of the dataframe.

In [ ]:
df['Rate']

##### Question 5: Get the average rate
Use the `.mean()` method to get the average rate and save it to a variable `average_rate`. Print it out to show what the average rate is

In [ ]:
average_rate = df['Rate'].mean()
average_rate

##### Question 6: Get the rows where `Rate` is null
Use the `.isna()` method with the `.loc` property to create a filter that returns rows where `Rate` is null. Show these rows of the dataframe.

In [ ]:
df.loc[ df['Rate'].isna() ]

##### Question 7: Get the indexes of the null rows
Use the `.index` property to get the indexes from the filtered dataframe and save them to a variable `null_rates`. Print out this variable as well.

In [ ]:
null_rates = df[ df['Rate'].isna() ].index
null_rates

##### Question 8: Get `Rate` column of filtered dataframe
Using the filtered dataframe from above (Question 6), get the `Rate` column by adding `Rate` to the `.loc` property.

You should get two rows back in a Series.

In [ ]:
df.loc[ df['Rate'].isna(), 'Rate' ]

##### Question 9: Set null rows equal to overall average rate
Using the code from the previous question, set the `Rate` of the two rows with null rates equal to the `average_rate`.

In [ ]:
df.loc[ df['Rate'].isna(), 'Rate' ] = average_rate

##### Question 10: Print out the number of nulls
Using the code from Question 2, print out the number of null values in each column.

In [ ]:
df.isna().sum()

#### Rate - Imputation (part 2)
##### Question 11: Is the overall average `Rate` the best?
Now we can see that the overall average `Rate` was successfully imputed into the null values for each `Rate`. However, is this `Rate` an accurate reflection of what the `Rate` likely actually is for both employees? What could be done to estimate the rate more accurately?

```
The overall average rate might be completely different from the actual average rate for each employee. Production Technicians and Sales Representatives might make more or less than the average employee. A more accurate representation of the actual likely rate for each one could be found by grouping the employees by department or job title and THEN finding the average rate in that group.
```

##### Question 12: Get `JobTitle` of each (previously) null row
Using the `.loc` property and the `null_rates` list of indexes that you obtained previously, print out the `JobTitle` column (a Series) for each of the rows which previously had a null value for `Rate`.

In [ ]:
df.loc[null_rates, 'JobTitle']

##### Question 13: Turn the Series into a list
Add the `.tolist()` method onto the previous line of code to place the two job titles into a list. Save this list of job titles into a variable `job_titles` and then print it out.

In [ ]:
job_titles = df.loc[null_rates, 'JobTitle'].tolist()
job_titles

##### Question 14: Filter the dataframe by `job_titles`
Using the `.isin()` method and the `.loc` property, get back all rows of the dataframe that have a value in the `JobTitle` column that exists in the `job_titles` list.

In [ ]:
df.loc[ df['JobTitle'].isin(job_titles)]

##### Question 15: Observe the filtered dataframe above
Observe the filtered dataframe returned above. What do you notice about the average rate of pay for the different job titles? How are they different from the overall average rate?

```
The overall average rate was $17.67. However, the average rate for a Product Technician is $15, and the average rate for a Sales Representative is $23.08. Thus, there is a fairly significant difference from the actual rates of people with those job titles and the overall average rate.
```

##### Question 16: Group by `JobTitle` and find the mode
Using the `.groupby()` and `.agg()` methods, group the filtered dataframe from question 13 by `JobTitle` and aggregate to find the most common value for `Rate`.

Note: Pandas doesn't have a built in aggregate function to find the mode using `.agg()`. Thus, the code `.agg({'Rate': 'mode'})` won't work. Instead, you'll have to use the Pandas Series method, which will look like this: `.agg({'Rate': pd.Series.mode})`.

In [ ]:
df.loc[ df['JobTitle'].isin(job_titles)].groupby('JobTitle').agg({'Rate': pd.Series.mode})

##### Question 17: Replace overall rates with average rate by job title
Using any way you would like, replace both of the (previously) null rates that were assigned the overall average rate a more precise average rate based on their `JobTitle`. You can either do this by using the `.loc` property and the indexes created before, or by using the `.loc` property with a filter by average `Rate` and a specific `JobTitle`.

It will be easiest to do this on more than one line.

In [ ]:
# Using the index created previously
df.loc[ null_rates[0], 'Rate'] = 15

In [ ]:
# Using a filter
df.loc[ (df['Rate'] == average_rate) & (df['JobTitle'] == 'Sales Representative'), 'Rate' ] = 23.0769

##### Question 18: Print out number of nulls
Using the `.isna()` and the `.sum()` methods, show how many null values exist in the dataframe.

In [ ]:
df.isna().sum()

#### SalariedFlag - Drop Column
The columns `SalariedFlag` and `EndDate` seem to have many null values. In a way, it makes sense that there might be null values in the `EndDate` column, since the database designers may have designed the system so that it doesn't record any value until an employee actually stops working at Adventure Works. However, the `SalariedFlag` field is a little more suspicious.

Most likely, we will end up dropping the `SalariedFlag` column because there are so many null values. However, let's make sure to look at it first to make sure that we *can't* impute it (ie. if we can see that all of the recorded values are the same, we might be able to justify imputation in specific situations).

##### Question 19: Print out counts of each unique value in `SalariedFlag`
Before anything else, we should check to see the counts of distinct values in the `SalariedFlag` column. Use the `.value_counts()` method to print this out.

In [ ]:
df['SalariedFlag'].value_counts()

##### Question 20: What do you see when counting the distinct values?
Upon observing the count of distinct values in the `SalariedFlag` column, what do you notice? Is there any way we could possibly justify imputation?

```
The counts of 1 and 0 are almost evenly split. It would be impossible to determine which rows should have a 1 or 0 without further evidence. For this reason, imputation cannot be used and the column should be dropped.
```

##### Question 21: Drop the `SalariedFlag` column
Use the `.drop()` method to drop the `SalariedFlag` column. 

In [ ]:
df.drop(columns='SalariedFlag', inplace=True)

##### Question 22: Print out the number of nulls
Use the `.isna()` and `.sum()` methods to show how many null values still exist in the dataframe. Make sure that the column `SalariedFlag` has been dropped successfully.

In [ ]:
df.isna().sum()

#### Check inconsistent format
To make sure that each of our columns has data in a consistent format, we should check to make sure that each column has the data type that we expect. If all of the quantitative fields are either `int64` or `float64`, we can move on to the categorical fields and get a count of each unique value to make sure each row has an expected value.

##### Question 23: Check for data types of quantitative fields
Use the `.info()` method to check the data types of each column in the dataframe.

In [ ]:
df.info()

##### Question 24: Does any column surprise you?
Looking at the information printed above and knowing a little about the data, do any of the column data types surprise you? Which column(s) surprise you and why? (yes, there should be at least one)

```
From what I have seen in previous data, the `CurrentFlag` column should be either a 1 or a 0 and thus have a Dtype of `int64`. However, it has a Dtype of `object`. Thus, I think that a string must have gotten introduced into the column's data.
```

##### Question 25: Check count of values of `CurrentFlag`
To be sure that there are issues with the `CurrentFlag` column, print out the count of unique values in the column using the `.value_counts()` method.

In [ ]:
df['CurrentFlag'].value_counts()

##### Question 26: What do you observe?
After counting up each unique value in the `CurrentFlag` column, what do you see?

```
There is a mix of 1s and the word "yes", which probably should have been 1s.
```

##### Question 27: Get rows where `CurrentFlag` is `yes`
Filter the dataframe using `.loc` to show only rows where `CurrentFlag` is equal to `"yes"`.

In [ ]:
df.loc[ df['CurrentFlag'] == "yes" ]

##### Question 28: Change each `"yes"` to `1`
It's probably safe to assume that, since all of the values in the `CurrentFlag` column are either `1` or `"yes"`, all `"yes"` values can be changed to 1. Use the `.replace()` method to replace all occurences of "yes" with **the integer** `1`. Save it to the original dataframe.

In [ ]:
df['CurrentFlag'].replace("yes", 1, inplace=True)

##### Question 29: Print out the value counts again
Use the `.value_counts()` method to print out the counts of unique values in the `CurrentFlag` again. Make sure that the only value that occurs now is `1`.

In [ ]:
df['CurrentFlag'].value_counts()

##### Question 30: What do you notice?
What do you notice about the results of the results of `.value_counts()`? Why do you think this occured?

```
The value `1` is counted twice as if there were two types of `1`. It must be because the first `1` is a string and the second `1` is an integer and so they are different.
```

#### Fix not matching data type
It looks like because the column `CurrentFlag` started out with strings inside of it, most of its values are still of data type `object`. Change it so that the `CurrentFlag` column has a data type of `int8`.

##### Question 31: Print out the data type of `CurrentFlag`
Using the `.dtype` property, print out the data type of the `CurrentFlag` column.

In [ ]:
df['CurrentFlag'].dtype

##### Question 32: Create `CurrentFlag` as data type `int8`
Using the `.astype()` method, print out a copy of the `CurrentFlag` column whose data type is `int8`.

In [ ]:
df['CurrentFlag'].astype('int8')

##### Question 33: Override the `CurrentFlag` column
Using the code from above, save the new `CurrentFlag` column of data type `int8` to the `CurrentFlag` column of the dataframe.

In [ ]:
df['CurrentFlag'] = df['CurrentFlag'].astype('int8')

##### Question 34: Print out the dataframe info
Use the `.info()` method to print out information about the data types of each column. Make sure that the `CurrentFlag` column now has a data type of `int8`.

In [ ]:
df.info()

#### Fix Extreme Values
While you have already fixed some columns with null values and incorrect values. Now, let's check the data set for extreme/incorrect values.

##### Question 35: Describe the dataframe
Use the `.describe()` method on the dataframe to get some summary statistics about each quantitative row.

In [ ]:
df.describe()

##### Question 36: What do you notice?
Observe the information created by the `.describe()` method above. What do you notice about the summary statistics that might indicate inaccurate data? You should see at least two columns with potential problems.

```
The `Rate` column has negative values, which is impossible and should be further investigated. The `VacationHours` column also has a very high maximum value and might be an error.
```

#### Fix Rate
##### Question 37: Get a filtered `Rate` column
Knowing that the `Rate` column has at least one negative number, create a filtered dataframe that contains rows where `Rate` is less than or equal to 0.

In [ ]:
df.loc[ df['Rate'] <= 0 ]

##### Question 38: What should be done?
Looking at the row(s) returned, what do you think should be done? Can the true `Rate` of the rows with negative values be imputed, or should they be dropped?

```
The `JobTitle` of the only row returned is Vice President of Sales, which makes me expect a higher rate. Simply making the `Rate` positive rather than negative makes sense to me.
```

##### Question 39: Make the negative `Rate` positive
Using the code from above, print out a the `Rate` column of rows whose `Rate` is less than or equal to 0. Multiply it by `-1` to make it positive.

In [ ]:
df.loc[ df['Rate'] <= 0, 'Rate' ] * -1

##### Question 40: Set the negative rate to positive rate
Using the code from above, filter the dataframe to get negative rates and set the new rates equal to the negative rate times `-1`.

In [ ]:
df.loc[ df['Rate'] <= 0, 'Rate' ] = df.loc[ df['Rate'] <= 0, 'Rate' ] * -1 

##### Question 41: Print out the dataframe description again
Use the `.describe()` method to print out the dataframe description again. Make sure that the minimum rate is greater than 1.

In [ ]:
df.describe()

#### Fix VacationHours
The maximum value in the `VacationHours` is 290-- that might be correct, but it also might be out of the ordinary. Let's do some work to try and fix it.

##### Question 42: Sort the dataframe by `VacationHours` descending
Use the `.sort_values()` method and the `by` and `ascending` arguments to print out the dataframe sorted by `VacationHours`, with the rows with the highest number of vacation hours on top. Add the `.head()` method after the `.sort_values()` method to get the top five rows.

In [ ]:
df.sort_values(by='VacationHours', ascending=False).head()

##### -- Check if outlier
##### Question 43: Import numpy
Before anything else, let's test to see whether `290` is really an outlier for the `VacationHours` column. To begin, import the NumPy library.

In [ ]:
import numpy as np

##### Question 44: Create function `isOutlier()`
Now create a function called `isOutlier()` that accepts one parameter `hours`. The function will be applied to each row of the `VacationHours` column and should perform the following:

1. Uses the `np.quantile()` function on the `VacationHours` column to get the first quartile (`0.25`) and save it to a variable `first_quartile`.
2. Uses the `np.quantile()` function on the `VacationHours` column to get the third quartile (`0.75`) and save it to a variable `third_quantile`.
3. Subtracts the `first_quartile` from the `third_quartile` and saves the results to a variable `iqr` (inner-quartile range).
4. Calculates the upper fence by performing `third_quartile + (1.5 * iqr)`, saving it to a variable `upper_fence`.
    - The upper fence is the upper limit for identifying outliers. Anything greater than the upper fence is considered an outlier.
5. If `hours` is greater than the `upper_fence`, return `True` (the hours for this row *is* an outlier). Otherwise, return `False`.

In [ ]:
def isOutlier(hours):
    # Get 25th percentile
    first_quartile = np.quantile(df['VacationHours'], 0.25)
    # Get 75th percentile
    third_quartile = np.quantile(df['VacationHours'], 0.75)
    # Get inner quartile range
    iqr = third_quartile - first_quartile
    # Calculate upper fence
    upper_fence = third_quartile + (1.5 * iqr)
    
    if hours > upper_fence:
        return True
    else:
        return False

##### Question 45: Apply function to the `VacationHours` column
Using the `.apply()` method, pass in the `isOutlier` function without parentheses to return a Series of True/False values. True indicates a row where `VacationHours` is an outlier.

In [ ]:
df['VacationHours'].apply(isOutlier)

##### Question 46: Save the Series above to a new column called `VacationHoursOutlier`
Using the code from above, create a new column in the dataframe called `VacationHoursOutlier` that has either a True or False if the `VacationHours` is an outlier.

In [ ]:
df['VacationHoursOutlier'] = df['VacationHours'].apply(isOutlier)

##### Question 47: Sort the dataframe by `VacationHours` descending again
In the same way that you did previously, use the `.sort_values()` method with the `by` and `ascending` arguments to view the rows with the highest number of `VacationHours`.

In [ ]:
df.sort_values(by='VacationHours', ascending=False).head()

##### Question 48: Is `290` an outlier for `VacationHours`?
Looking at the dataframe returned above, is `290` an outlier in the `VacationHours` column? If so, what could be done to fix this value?

```
The number `290` is an outlier in the `VacationHours` column. This row could be dropped, or we could use imputation to determine what other employees with the same job title have as their `VacationHours` and assign an average value to the row.
```

##### Question 49: Get rows of employees with `JobTitle` Sales Representative
As seen above, the row with total `VacationHours` has a `JobTitle` of "Sales Representative". Use a filter to get back rows of employees whose `JobTitle` is also "Sales Representative".

In [ ]:
df.loc[ df['JobTitle'] == 'Sales Representative' ]

##### Question 50: Remove the row with outliers from the filter
Add to the filter that you created above so that the row with an outlier in the `VacationHours` field is not included in the filtered dataframe.

In [ ]:
df.loc[ (df['JobTitle'] == 'Sales Representative') & (df['VacationHours'] != 290) ]

##### Question 51: Get the average vacation hours among Sales Representatives
Using the filtered dataframe above, get the average `VacationHours` for Sales Representatives. Store this number in a variable called `sales_rep_average_vac_hours` and then print it out.

In [ ]:
sales_rep_average_vac_hours = df.loc[ (df['JobTitle'] == 'Sales Representative') & (df['VacationHours'] != 290), 'VacationHours' ].mean()
sales_rep_average_vac_hours

##### Question 52: Assign the average vacation hours to the employee with an outlier `VacationHours`
Using the `.loc` property, change the `VacationHours` of the employee with an outlier in the `VacationHours` column to be `sales_rep_average_vac_hours` instead.

In [ ]:
df.loc[ df['VacationHoursOutlier'], 'VacationHours'] = sales_rep_average_vac_hours

##### Question 53: Print out the dataframe description
Finally, print out the dataframe description using the `.describe()` method. Make sure the maximum value for `VacationHours` is no longer 290.

In [ ]:
df.describe()

#### Final Thoughts
In case you were curious, the value of the row with an outlier in `VacationHours` was originally `29` instead of `290`. That means that the error was most likely caused by a human who accidentally entered in an extra `0` on the end of the actual number. In any case, the average vacation hours among Sales Representatives turned out to be fairly close to this (`31.15`), meaning that imputation worked fairly well in this example.